# PART 1: Connect to the SQLite IMDB Database

To establish a connection with the SQLite IMDB database, the `imdb-sqlite` Python module is utilized. If you haven't installed it yet, you can do so by running the following command:

```bash
pip install imdb-sqlite
```
For detailed information and documentation about the imdb-sqlite module, please refer to the official PyPI page: [Link Text](https://pypi.org/project/imdb-sqlite/).

This module not only facilitates the connection to the IMDB database but also streamlines the process by automatically renaming columns and adding relevant indexers.

Please follow the steps in the documentation how to download the dataset into a db file.

In [1]:
import sqlite3

# Replace 'your_database_file.db' with the actual name of your SQLite database file
db_file = 'imdb.db'


def execute_query(query: str):
    connection = sqlite3.connect(db_file)
    # Create a cursor object to execute SQL queries
    cursor = connection.cursor()
    # Execute the query
    cursor.execute(query)

    results = cursor.fetchall()
    for result in results:
        print(result)

    # Close the cursor and the connection
    cursor.close()
    connection.close()

Verify that the tables are successfully loaded

In [2]:
table_query = "SELECT name FROM sqlite_master WHERE type='table';"

execute_query(table_query)

('people',)
('titles',)
('akas',)
('crew',)
('episodes',)
('ratings',)
('sqlite_stat1',)


# PART 2: Analyzing IMDB Data with SQL Queries

In this section, we aim to obtain responses to the following questions using SQL queries. These responses will serve as valuable benchmarks for future comparisons with the assistance of the Language Model (LLM):

1. **How many movies were released in the last year?**
   
2. **How many actors were born in the year 2001?**
   
3. **What is the language of the movie "Pulp Fiction"?**
   
4. **What is the series with the most episodes?**
   
5. **In how many horror movies did Sean Connery perform?**



Some index that help for performance

In [ ]:
connection = sqlite3.connect(db_file)
cursor = connection.cursor()


cursor.execute("CREATE INDEX idx_people_born ON people(born);")
cursor.execute("CREATE INDEX idx_people_person_id ON people(person_id);")
cursor.execute("CREATE INDEX idx_crew_person_id ON crew(person_id);")
cursor.execute("CREATE INDEX idx_crew_category ON crew(category);")
cursor.execute("CREATE INDEX idx_titles_title_id ON titles(title_id);")
cursor.execute("CREATE INDEX idx_episodes_show_title_id ON episodes(show_title_id);")
cursor.execute("CREATE INDEX idx_titles_type ON titles(type);")

# Commit the changes
connection.commit()

In [4]:
movies_count_query = """
    SELECT count(*) FROM titles t 
    WHERE  t.premiered = 2023 AND type = "movie"
"""
execute_query(movies_count_query)


(18337,)


In [20]:

actors_count_query = """
    SELECT COUNT(*) FROM people 
    WHERE born = 2001 AND person_id IN (SELECT DISTINCT person_id FROM crew WHERE category = 'actor')
"""

execute_query(actors_count_query)


539


In [31]:
pulp_fiction_query = """
    SELECT akas.language 
    FROM akas 
    INNER JOIN titles ON akas.title_id = titles.title_id 
    WHERE titles.primary_title = "Pulp Fiction" 
        AND titles.type = "movie" 
        AND akas.is_original_title = 1;
"""
execute_query(pulp_fiction_query)


(None,)


After inspection, we saw that the "Pulp Fiction" movie has a None value in its language field. This indicates that we are unable to retrieve the original language of the movie from the available data. The most values of the language column is missing.

In [32]:
pulp_fiction_query = """
    SELECT * 
    FROM akas 
    INNER JOIN titles ON akas.title_id = titles.title_id 
    WHERE titles.primary_title = "Pulp Fiction" AND titles.type = "movie" AND akas.is_original_title = 1;
"""
execute_query(pulp_fiction_query)

('tt0110912', 'Pulp Fiction', None, None, 'original', None, 1, 'tt0110912', 'movie', 'Pulp Fiction', 'Pulp Fiction', 0, 1994, None, 154, 'Crime,Drama')


In [33]:
# SQL query to count the number of episodes for each TV series and retrieve the top 5
series_query = """
    SELECT titles.primary_title, COUNT(episodes.episode_title_id) AS num_episodes
    FROM titles
    INNER JOIN episodes ON titles.title_id = episodes.show_title_id
    WHERE titles.type = "tvSeries"
    GROUP BY titles.primary_title
    ORDER BY num_episodes DESC
    LIMIT 5;
"""

execute_query(series_query)
  

('NRK Nyheter', 18593)
('Days of Our Lives', 14884)
('The Young and the Restless', 13046)
('General Hospital', 12846)
('Coronation Street', 10761)


In [34]:

sean_connery_query = """
SELECT count(*) FROM titles t 
INNER JOIN crew c ON t.title_id = c.title_id 
INNER JOIN people p ON c.person_id = p.person_id 
WHERE  p.name = "Sean Connery" AND t.type = "movie" AND t.genres like "%Horror%"

"""

execute_query(sean_connery_query)

(0,)


# Summary of Query Responses

1. **How many movies were released in the last year?**
   - **Response:** 18337

2. **How many actors were born in the year 2001?**
   - **Response:** 539

3. **What is the language of the movie "Pulp Fiction"?**
   - **Response:** Missing data

4. **What is the series with the most episodes?**
   - **Response:**
     - ('NRK Nyheter', 18593)
     - ('Days of Our Lives', 14884)
     - ('The Young and the Restless', 13046)
     - ('General Hospital', 12846)
     - ('Coronation Street', 10761)

5. **In how many horror movies did Sean Connery perform?**
   - **Response:** 0


# PART 3: Utilizing Langchain with 'SQLDatabaseChain' for Querying Plain Text Questions

In this section, we explore the integration of Langchain, specifically utilizing the 'SQLDatabaseChain' for obtaining responses from plain text questions.

In [3]:
from langchain_community.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain
from langchain_openai import OpenAI


db = SQLDatabase.from_uri("sqlite:///imdb.db")
# the default model used: gpt-3.5-turbo-instruct
llm = OpenAI(api_key='{your_API_key}',temperature=0, verbose=True)
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)

In [6]:
questions = {"How many movies were released in the last year?", 
             "How many actors were born in the year 2001?", 
             "What is the language of the movie 'Pulp Fiction'?", 
             "What is the series with the most episodes?", 
             "In how many horror movies did Sean Connery perform?"}

for question in questions:
    result = db_chain.invoke(question)
    print(result['result'])



> Entering new SQLDatabaseChain chain...
What is the series with the most episodes?
SQLQuery:SELECT titles.primary_title, COUNT(episodes.episode_title_id) AS num_episodes
FROM titles
INNER JOIN episodes ON titles.title_id = episodes.show_title_id
WHERE titles.type = "tvSeries"
GROUP BY titles.primary_title
ORDER BY num_episodes DESC
LIMIT 1;
SQLResult: [('NRK Nyheter', 18593)]
Answer:NRK Nyheter
> Finished chain.
NRK Nyheter


> Entering new SQLDatabaseChain chain...
In how many horror movies did Sean Connery perform?
SQLQuery:SELECT COUNT(*) FROM titles t
INNER JOIN ratings r ON t.title_id = r.title_id
INNER JOIN crew c ON t.title_id = c.title_id
INNER JOIN people p ON c.person_id = p.person_id
WHERE p.name = "Sean Connery" AND t.genres LIKE "%Horror%"
SQLResult: [(0,)]
Answer:0
> Finished chain.
0


> Entering new SQLDatabaseChain chain...
What is the language of the movie 'Pulp Fiction'?
SQLQuery:SELECT akas.language FROM akas INNER JOIN titles ON akas.title_id = titles.title_id W

# Analysis of LLM Query Responses

In reviewing the results of the queries generated by the Language Model (LLM), we observed commendable accuracy in converting plain text to SQL. However, there are instances where the LLM provided incorrect responses. Let's delve into the reasons and propose improvements:

## 1. "How many movies were released in the last year?"

The LLM appears to assume the current year is 2023 instead of 2024, impacting the interpretation of the query.

- **Cause:** Lack of connection to the timeline.
- **Improvement:** Instruct the LLM to use the current date function when the question is relevant to time.

## 2. "How many actors were born in the year 2001?"

The LLM fails to specify that the query is specifically about actors, indicating a potential information gap in the crew table schema.

- **Cause:** Insufficient information in the crew table schema.
- **Improvement:** Enhance the prompt by providing more details about the crew table, such as possible values for the Category column.

## 3. "What is the language of the movie 'Pulp Fiction'?"

While the LLM gives a correct answer, examining the generated SQL reveals a lack of precision regarding that it not used the column is_original_title.

- **Cause:** Inadequate information in the titles table schema.
- **Improvement:** Enrich the prompt with additional details about the titles table, including possible values for the type column and explanation about the is_original_title column.

By addressing these specific causes and making prompt enhancements, we can guide the LLM toward more accurate and context-aware responses.

-----------------------------------------------------------------------------------------

# PART 4: Improve the LLM 

In this section, we will explore modifications to the Language Model's prompt to provide additional information, thereby improving its ability to generate accurate answers.

Let's retrieve the distinct values possible for the 'Category' column in the 'Crew' table.

In [10]:
category_query = "SELECT DISTINCT category FROM crew"

# Execute the query
execute_query(category_query)

('actor',)
('actress',)
('archive_footage',)
('archive_sound',)
('cinematographer',)
('composer',)
('director',)
('editor',)
('producer',)
('production_designer',)
('self',)
('writer',)


Let's retrieve the distinct values possible for the 'type' column in the 'titles' table.

In [11]:
type_query = "SELECT DISTINCT type FROM titles"

# Execute the query
execute_query(type_query)

('movie',)
('short',)
('tvEpisode',)
('tvMiniSeries',)
('tvMovie',)
('tvPilot',)
('tvSeries',)
('tvShort',)
('tvSpecial',)
('video',)
('videoGame',)


# Steps to Improve the LLM

To enhance the precision of the Language Model (LLM), I have implemented the following steps:

1. **Retrieve Prompt Template:**
   - I extracted the prompt template from the Langchain code to better understand the existing structure.

2. **Modify Table Information:**
   - I made adjustments to the information available to the LLM by adding the following lines to the 'crew' and 'titles' tables:
     ```sql
     category VARCHAR CHECK (category IN ('actor', 'actress', 'archive_footage', 'archive_sound', 'cinematographer', 'composer', 'director', 'editor', 'producer', 'production_designer', 'self', 'writer'))
     type VARCHAR CHECK (type IN ('movie', 'short', 'tvEpisode', 'tvMiniSeries', 'tvMovie', 'tvPilot', 'tvSeries', 'tvShort', 'tvSpecial', 'video', 'videoGame'))
     ```
     These additions aim to provide the model with a better understanding of the data.

3. **Incorporate Date Information:**
   - To ensure the LLM is aware of the current date, I utilized the 'partial_variables' parameter, passing it the `_get_datetime()` method.
   - Additionally, I included the following in the prompt: `'The current date is {date}.'`, guiding the LLM to utilize the provided method.

By implementing these steps, we aim to equip the LLM with more contextual information, enhancing its ability to generate accurate and relevant responses.


In [8]:
from langchain.prompts import PromptTemplate

from datetime import datetime


def _get_datetime():
    now = datetime.now()
    return now.strftime("%m/%d/%Y, %H:%M:%S")

PROMPT_SUFFIX = """Only use the following tables:
CREATE TABLE akas (
	title_id VARCHAR, 
	title VARCHAR, 
	region VARCHAR, 
	language VARCHAR, 
	types VARCHAR, 
	attributes VARCHAR, 
	is_original_title INTEGER -- this column is to describe if the title is the original language title,  possible values: This column has two possible values, 0 or 1
)

/*
3 rows from akas table:
title_id	title	region	language	types	attributes	is_original_title
tt0000001	Карменсіта	UA	None	imdbDisplay	None	0
tt0000001	Carmencita	DE	None	None	literal title	0
tt0000001	Carmencita - spanyol tánc	HU	None	imdbDisplay	None	0
*/


CREATE TABLE crew (
	title_id VARCHAR, 
	person_id VARCHAR, 
	category VARCHAR CHECK (category IN ('actor', 'actress', 'archive_footage', 'archive_sound', 'cinematographer', 'composer', 'director', 'editor', 'producer', 'production_designer', 'self', 'writer')), 
	job VARCHAR, 
	characters VARCHAR
)

/*
3 rows from crew table:
title_id	person_id	category	job	characters
tt0000001	nm1588970	self	None	["Self"]
tt0000001	nm0005690	director	None	None
tt0000001	nm0374658	cinematographer	director of photography	None
*/


CREATE TABLE episodes (
	episode_title_id VARCHAR, 
	show_title_id VARCHAR, 
	season_number INTEGER, 
	episode_number INTEGER
)

/*
3 rows from episodes table:
episode_title_id	show_title_id	season_number	episode_number
tt0041951	tt0041038	1	9
tt0042816	tt0989125	1	17
tt0042889	tt0989125	None	None
*/


CREATE TABLE people (
	person_id VARCHAR, 
	name VARCHAR, 
	born INTEGER, 
	died INTEGER, 
	PRIMARY KEY (person_id)
)

/*
3 rows from people table:
person_id	name	born	died
nm0000001	Fred Astaire	1899	1987
nm0000002	Lauren Bacall	1924	2014
nm0000003	Brigitte Bardot	1934	None
*/


CREATE TABLE ratings (
	title_id VARCHAR, 
	rating INTEGER, 
	votes INTEGER, 
	PRIMARY KEY (title_id)
)

/*
3 rows from ratings table:
title_id	rating	votes
tt0000001	5.7	2014
tt0000002	5.7	270
tt0000003	6.5	1937
*/


CREATE TABLE titles (
	title_id VARCHAR, 
	type VARCHAR CHECK (type IN ('movie', 'short', 'tvEpisode', 'tvMiniSeries', 'tvMovie', 'tvPilot', 'tvSeries', 'tvShort', 'tvSpecial', 'video', 'videoGame')), 
	primary_title VARCHAR, 
	original_title VARCHAR, 
	is_adult INTEGER, 
	premiered INTEGER, 
	ended INTEGER, 
	runtime_minutes INTEGER, 
	genres VARCHAR, 
	PRIMARY KEY (title_id)
)

/*
3 rows from titles table:
title_id	type	primary_title	original_title	is_adult	premiered	ended	runtime_minutes	genres
tt0000001	short	Carmencita	Carmencita	0	1894	None	1	Documentary,Short
tt0000002	short	Le clown et ses chiens	Le clown et ses chiens	0	1892	None	5	Animation,Short
tt0000003	short	Pauvre Pierrot	Pauvre Pierrot	0	1892	None	4	Animation,Comedy,Romance
*/

Question: {input}"""

_sqlite_prompt = """You are a SQLite expert. Given an input question, first create a syntactically correct SQLite query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per SQLite. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
The current date is {date}.

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of the SQLQuery
Answer: Final answer here

"""
#Pay attention to use the {date} function to get the current date if the question involves working with date and time information or requires referencing the current date in your SQL queries.

SQLITE_PROMPT = PromptTemplate(
    input_variables=["input", "top_k"],
    template=_sqlite_prompt + PROMPT_SUFFIX, partial_variables={"date": _get_datetime}
)

In [9]:
db_chain_with_custom_prompt = SQLDatabaseChain.from_llm(llm, db, verbose=True,return_intermediate_steps=True, prompt=SQLITE_PROMPT)

### Now, we will re-run the process using the custom template, and anticipate improved accuracies for the questions

In [84]:
questions = {"How many movies were released in the last year?", 
             "How many actors were born in the year 2001?", 
             "What is the language of the movie 'Pulp Fiction'?", 
             "What is the series with the most episodes?", 
             "In how many horror movies did Sean Connery perform?"}

for question in questions:
    result = db_chain_with_custom_prompt.invoke(question)
    print(result['result'])



> Entering new SQLDatabaseChain chain...
What is the series with the most episodes?
SQLQuery:SELECT titles.primary_title, COUNT(episodes.episode_title_id) AS num_episodes
FROM titles
INNER JOIN episodes ON titles.title_id = episodes.show_title_id
WHERE titles.type = 'tvSeries'
GROUP BY titles.primary_title
ORDER BY num_episodes DESC
LIMIT 1;
SQLResult: [('NRK Nyheter', 18593)]
Answer:NRK Nyheter
> Finished chain.
NRK Nyheter


> Entering new SQLDatabaseChain chain...
In how many horror movies did Sean Connery perform?
SQLQuery:SELECT COUNT(*) FROM titles t
INNER JOIN ratings r ON t.title_id = r.title_id
INNER JOIN crew c ON t.title_id = c.title_id
INNER JOIN people p ON c.person_id = p.person_id
WHERE t.genres LIKE '%Horror%' AND p.name = 'Sean Connery' AND c.category IN ('actor', 'actress')
SQLResult: [(0,)]
Answer:0
> Finished chain.
0


> Entering new SQLDatabaseChain chain...
What is the language of the movie 'Pulp Fiction'?
SQLQuery:SELECT akas.language FROM akas INNER JOIN titl

Regarding the question "What is the language of the movie 'Pulp Fiction'?" it appears that the use of the column 'is_original_title' is still being avoided. 

My interpretation is that the question might not be well-written. It would be clearer to specify 'the original language' instead. For example, rephrasing the question as "What is the original language of the movie 'Pulp Fiction'?" would likely improve clarity.

In [11]:
result = db_chain_with_custom_prompt.invoke('What is the original language of the movie "Pulp Fiction"?')
print(result['result'])



> Entering new SQLDatabaseChain chain...
What is the original language of the movie "Pulp Fiction"?
SQLQuery:SELECT akas.language FROM akas INNER JOIN titles ON akas.title_id = titles.title_id WHERE titles.original_title = "Pulp Fiction" AND akas.is_original_title = 1 LIMIT 5
SQLResult: [(None,)]
Answer:None
> Finished chain.
None




Initially, I attempted to utilize the SQL agent, attracted by its greater flexibility and advantages as described here: : https://python.langchain.com/docs/use_cases/qa_structured/sql#case-3-sql-agents. 

However, I experienced subpar results when compared to the SQLDatabaseChain. Also managing and updating prompts proved to be more complex with the SQL agent. As a result, I decided to continue with the 'SQLDatabaseChain.

In [15]:
from langchain.agents import create_sql_agent

# from langchain.agents import AgentExecutor
from langchain.agents.agent_types import AgentType
from langchain_community.agent_toolkits import SQLDatabaseToolkit

db1 = SQLDatabase.from_uri("sqlite:///imdb.db")

agent_executor = create_sql_agent(
    llm=llm,
    toolkit=SQLDatabaseToolkit(db=db1, llm=llm),
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True
)


In [66]:
agent_executor.invoke("What is the language of the movie 'Pulp Fiction'?")



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: 
Observation: akas, crew, episodes, people, ratings, titles
Thought: I should query the schema of the titles table to see what columns I can use.
Action: sql_db_schema
Action Input: titles
Observation: 
CREATE TABLE titles (
	title_id VARCHAR, 
	type VARCHAR, 
	primary_title VARCHAR, 
	original_title VARCHAR, 
	is_adult INTEGER, 
	premiered INTEGER, 
	ended INTEGER, 
	runtime_minutes INTEGER, 
	genres VARCHAR, 
	PRIMARY KEY (title_id)
)

/*
3 rows from titles table:
title_id	type	primary_title	original_title	is_adult	premiered	ended	runtime_minutes	genres
tt0000001	short	Carmencita	Carmencita	0	1894	None	1	Documentary,Short
tt0000002	short	Le clown et ses chiens	Le clown et ses chiens	0	1892	None	5	Animation,Short
tt0000003	short	Pauvre Pierrot	Pauvre Pierrot	0	1892	None	4	Animation,Comedy,Romance
*/
Thought: I should query the primary_title column from the titles table for the movie 'Pulp Fiction'.
Action

{'input': "What is the language of the movie 'Pulp Fiction'?",
 'output': 'Pulp Fiction'}